In [2]:
pip install fastapi uvicorn python-multipart aiofiles


In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
import numpy as np
import tensorflow as tf
from io import BytesIO
from PIL import Image
import cv2

app = FastAPI()

# Load your trained model
model_path = 'best_model.keras'
model = tf.keras.models.load_model(model_path)

def read_image(file) -> np.array:
    image = Image.open(BytesIO(file))
    image = image.convert('L')  # Convert to grayscale
    image = image.resize((256, 256))  # Resize to match model input
    image = np.array(image)
    image = np.expand_dims(image, axis=-1)
    image = np.expand_dims(image, axis=0)
    image = image / 255.0  # Normalize image
    return image

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    image = await file.read()
    image_array = read_image(image)
    predictions = model.predict(image_array)
    result = predictions[0].tolist()  # Convert to list for JSON response
    return JSONResponse(content={"prediction": result})


In [ ]:
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
